# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.dungeon.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState

player = DungeonPlayerState(
    playerId="TEST0",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,        
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="스킬 사용법이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[1001],
}
await graph.ainvoke(inputs, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("저 보스몬스터는 뭐야?")]
}, config=config)

result = await graph.ainvoke({
    "messages":[add_human_message("내가 쓸만한 아이템 뭐있어?")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


총 질문 <Used Abilities>
['USAGE_GUIDE']
</Used Abilities>

<Heroine Info>
{
  "playerId": "TEST0",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": {
    "weaponId": 22,
    "weaponType": 2,
    "weaponName": "레어 쌍검",
    "rarity": 2,
    "attackPower": 11,
    "staggerPower": 1,
    "modifier": {
      "strength": 0.1,
      "dexterity": 0.8,
      "intelligence": 0.1
    }
  },
  "inventory": [
    0,
    21
  ]
}
</Heroine Info>

<Current Situation>
    <USAGE_GUIDE>

# Game System Overview

This document summarizes the core game mechanics, including the dungeon-exploration-focused gameplay loop, character controls, and interaction systems with heroines and the 현자 NPC.

---

# 1. Core Gameplay Loop

The game progresses through the following repeating cycle.

## 1-1. Dungeon Exploration
- The player explores **roguelike-style dungeons**.
- Through combat, the player uses skills, strengthens weapons and equipment,

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [ ]:
result = await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?"),
     "dungenon_player": player,
     "target_monster_ids":[0,1],
     }, config=config
)   

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="다음방에 뭐있는지 확인해봐"),
        "dungenon_player": player,
    },
    config=config,
)
result["messages"]

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [ ]:
player = DungeonPlayerState(
    playerId="TEST0",
    heroineId=1,
    currRoomId=3,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },

    config=config,
)
result["messages"]

# 전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
# expected_outcome='1번 선택은 기억의 고통을 정면으로 마주해 정신적 타격(instant_damage_medium)을 입지만, 트라우마를 전투 감각으로 전환해 스킬 데미지(skill_damage_increase_all)가 크게 상승한다.\n2번 선택은 죄책감을 회피해 즉각적인 피해는 피하지만, 마음이 무거운 족쇄가 되어 이동속도(move_speed_decrease_all)가 감소해 전투와 탐사에서 기동력이 떨어진다.\n3번 선택은 분노로 정신을 끊어내며 일시적 피해(instant_damage_low)를 받는 대신, 신경이 과도하게 흥분해 공격 속도(attack_speed_increase_all)가 빨라져 근접 교전에 유리해진다.\n4번 선택은 죄책감을 인정하고 상처를 들여다보며 내면이 단단해져 최대 체력(hp_increase_all)이 상승하지만, 기억의 잔향이 스킬 운용을 방해해 일정 시간 스킬 데미지 감소(curse_debuff)를 겪게 된다.') floor=1



## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [ ]:
player = DungeonPlayerState(
    playerId="TEST0",
    heroineId=1,
    currRoomId=2,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이 던전에 몬스터 뭐 나와?"),
        "dungenon_player": player,
    },
    config=config,
)
result["dungenon_player"]
result["messages"]
# 전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>, <FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]


## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?"),
     "dungenon_player": player, },
    config=config
)
result["messages"]

In [ ]:
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="그럼 지금 보스방은 못가?"),
#      "dungenon_player": player, },
#     config=config
# )
result = await graph.ainvoke(
    { "messages": add_human_message(content="보스방울 어떻게 봐?"),
     "dungenon_player": player, },
    config=config
)


In [ ]:
result["messages"][-1]

In [ ]:
from agents.fairy.interaction.fairy_interaction_agent import graph_builder as interaction_builder
interaction_graph = interaction_builder.compile() 
interaction_graph.invoke({
    "messages": add_human_message(content="다음방 확인해"),
    "inventory": [0, 21, 42],
})

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.guild.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
guild_graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
result = await guild_graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는 힌트좀 줘") },
    config=config
)
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
#     config=config
# )
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:656: SyntaxWarning: invalid escape sequence '\-'
  "info": """
/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:750: SyntaxWarning: invalid escape sequence '\-'
  "info": """
/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/cache_data.py:839: SyntaxWarning: invalid escape sequence '\+'
  "info": """
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.val

[HumanMessage(content='히로인의 호감도 얻는 힌트좀 줘', additional_kwargs={'created_at': '2025-12-16T17:22:53.866992'}, response_metadata={}, id='6b76f5ef-c68c-42a9-9128-8d0c4e432c7d'),
 AIMessage(content='로코는 따뜻한 열기, 머리 쓰다듬기, 금속 소리, 정직함을 좋아해요! 그걸 활용해보세요~', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 3021, 'total_tokens': 3478, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 420, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 304, 'text_tokens': 3021, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_cfcc85ff86', 'id': 'd3879171-e735-2913-3249-7e6fe713cd0e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2641-5459-7c81-8953-a73f0cc226d0-0', usage_metadata={'input_tokens': 3021, 'output_tokens': 37, 'total_tokens': 3478, 'input_token_

In [ ]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="왜 던전에서 사냥해야해? 이유가 있어?") },
    config=config
)

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


{'messages': [HumanMessage(content='히로인의 호감도 얻는 힌트좀 줘', additional_kwargs={'created_at': '2025-12-16T17:22:53.866992'}, response_metadata={}, id='6b76f5ef-c68c-42a9-9128-8d0c4e432c7d'),
  AIMessage(content='로코는 따뜻한 열기, 머리 쓰다듬기, 금속 소리, 정직함을 좋아해요! 그걸 활용해보세요~', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 3021, 'total_tokens': 3478, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 420, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 304, 'text_tokens': 3021, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_cfcc85ff86', 'id': 'd3879171-e735-2913-3249-7e6fe713cd0e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2641-5459-7c81-8953-a73f0cc226d0-0', usage_metadata={'input_tokens': 3021, 'output_tokens': 37, 'total_tokens': 3478,

In [3]:
await guild_graph.ainvoke(
    { "messages": add_human_message(content="나 이제 뭐 해야돼?") },
    config=config
)

{'messages': [HumanMessage(content='히로인의 호감도 얻는 힌트좀 줘', additional_kwargs={'created_at': '2025-12-16T17:22:53.866992'}, response_metadata={}, id='6b76f5ef-c68c-42a9-9128-8d0c4e432c7d'),
  AIMessage(content='로코는 따뜻한 열기, 머리 쓰다듬기, 금속 소리, 정직함을 좋아해요! 그걸 활용해보세요~', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 3021, 'total_tokens': 3478, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 420, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 304, 'text_tokens': 3021, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_cfcc85ff86', 'id': 'd3879171-e735-2913-3249-7e6fe713cd0e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2641-5459-7c81-8953-a73f0cc226d0-0', usage_metadata={'input_tokens': 3021, 'output_tokens': 37, 'total_tokens': 3478,

In [5]:
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 100,
                    " affection":100, "sanity": 100, "thread_id": "100"}
}
await guild_graph.ainvoke(
    { "messages": add_human_message(content="모든 메모리를 해제했엉 이제 나 뭐해야해?") },
    config=config
)

{'messages': [HumanMessage(content='히로인의 호감도 얻는 힌트좀 줘', additional_kwargs={'created_at': '2025-12-16T17:22:53.866992'}, response_metadata={}, id='6b76f5ef-c68c-42a9-9128-8d0c4e432c7d'),
  AIMessage(content='로코는 따뜻한 열기, 머리 쓰다듬기, 금속 소리, 정직함을 좋아해요! 그걸 활용해보세요~', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 3021, 'total_tokens': 3478, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 420, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 304, 'text_tokens': 3021, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_cfcc85ff86', 'id': 'd3879171-e735-2913-3249-7e6fe713cd0e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2641-5459-7c81-8953-a73f0cc226d0-0', usage_metadata={'input_tokens': 3021, 'output_tokens': 37, 'total_tokens': 3478,